In [3]:
# importamos librerias de pyspark para realizar el preprocesado de los datos
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, isnan, isnull, mean, round
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidatorModel
from pyspark.ml.feature import IndexToString
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# importamos funciones auxiliares
# from filter_datasets import *
# from process_2008_data import *
from models import *


In [4]:
# se crea la sesion de spark
spark = SparkSession.builder.appName("proyecto").getOrCreate()


In [ ]:
file_configs = [
    {"input": "airports.csv", "output": "filtered_airports.csv", "columns": ["iata"]},
    {"input": "carriers.csv", "output": "filtered_carriers.csv", "columns": ["Code"]},
    {
        "input": "plane-data.csv",
        "output": "filtered_plane_data.csv",
        "columns": ["tailnum"],
    },
]

# Process each file
for config in file_configs:
    filter_columns(config["input"], config["output"], config["columns"])

In [ ]:
# Input and output file paths
input_2008_file = "2008.csv"
input_plane_file = "plane-data.csv"
output_file = "processed_2008.csv"
#original_col = [Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay]
# Run the function
process_2008_data(input_2008_file, input_plane_file, output_file)

In [12]:
# EDA
# se carga el dataset
df = spark.read.csv("processed_2008.csv", header=True, inferSchema=True)
cols = (
    df.columns
)  # Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,TailNum,IssueDate,CRSElapsedTime,ArrDelay,DepDelay,Origin,Dest,Cancelled

target = "ArrDelay"
df.printSchema()

root
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: double (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- IssueDate: string (nullable = true)
 |-- CRSElapsedTime: double (nullable = true)
 |-- ArrDelay: double (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: integer (nullable = true)



In [13]:
# comprueba si hay valores nulos
for col in cols:
    print(col, df.filter(df[col].isNull()).count())


Month 0
DayofMonth 0
DayOfWeek 0
DepTime 0
CRSDepTime 0
ArrTime 0
CRSArrTime 0
UniqueCarrier 0
TailNum 0
IssueDate 192053
CRSElapsedTime 0
ArrDelay 0
DepDelay 0
Origin 0
Dest 0
Cancelled 0


In [14]:
# elimeinamos los nulos en la columna ArrDelay
df = df.filter(df["ArrDelay"].isNotNull())

In [15]:
# comprueba si hay valores nulos
for col in cols:
    print(col, df.filter(df[col].isNull()).count())

Month 0
DayofMonth 0
DayOfWeek 0
DepTime 0
CRSDepTime 0
ArrTime 0
CRSArrTime 0
UniqueCarrier 0
TailNum 0
IssueDate 192053
CRSElapsedTime 0
ArrDelay 0
DepDelay 0
Origin 0
Dest 0
Cancelled 0


In [21]:
# Se crea un vector con las columnas que se van a utilizar
features = df.columns
features.remove("ArrDelay")
from pyspark.sql.functions import col
# Se convierte la variable target a numérica
df = df.withColumn("ArrDelay", df["ArrDelay"].cast(IntegerType()))
df = df.withColumn("ArrDelay", when(col("ArrDelay") > 15, 1).otherwise(0))
df = df.withColumn("ArrDelay", df["ArrDelay"].cast(IntegerType()))

# Se convierten las variables categóricas a numéricas (TailNum, UniqueCarrier, Origin, Dest)
indexers = [
    StringIndexer(inputCol=column, outputCol=column + "_index").fit(df)
    for column in ["UniqueCarrier", "TailNum", "Origin", "Dest"]
]

pipeline = Pipeline(stages=indexers)
df = pipeline.fit(df).transform(df)

# Mostrar las primeras filas del dataframe




IllegalArgumentException: requirement failed: Output column UniqueCarrier_index already exists.

In [22]:
print("Primeras filas del dataframe:")
df.show(5)

Primeras filas del dataframe:
+-----+----------+---------+-------+----------+-------+----------+-------------+-------+----------+--------------+--------+--------+------+----+---------+-------------------+-------------+------------+----------+
|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|TailNum| IssueDate|CRSElapsedTime|ArrDelay|DepDelay|Origin|Dest|Cancelled|UniqueCarrier_index|TailNum_index|Origin_index|Dest_index|
+-----+----------+---------+-------+----------+-------+----------+-------------+-------+----------+--------------+--------+--------+------+----+---------+-------------------+-------------+------------+----------+
|    1|         3|        4| 1343.0|      1325| 1451.0|      1435|           WN| N240WN|07/11/2006|          70.0|       0|    18.0|   HOU| LIT|        0|                0.0|        562.0|        39.0|      79.0|
|    1|         3|        4| 1125.0|      1120| 1247.0|      1245|           WN| N523SW|02/11/1992|          85.0|    

In [23]:
# eliminamos las filas que hemos indexado
df = df.drop(*["UniqueCarrier", "TailNum", "Origin", "Dest"])

In [27]:
# linear regression
# Uso de funciones del archivo models.py
from models import estimate_lr, estimate_kmeans
input_columns_lr = [col for col in df.columns if (col != "ArrDelay" and col != "IssueDate")]
print(input_columns_lr)
model = estimate_lr(df, input_columns_lr, target)

['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime', 'CRSElapsedTime', 'DepDelay', 'Cancelled', 'UniqueCarrier_index', 'TailNum_index', 'Origin_index', 'Dest_index']
+-----+----------+---------+-------+----------+-------+----------+----------+--------------+--------+--------+---------+-------------------+-------------+------------+----------+--------------------+--------------------+----------+
|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime| IssueDate|CRSElapsedTime|ArrDelay|DepDelay|Cancelled|UniqueCarrier_index|TailNum_index|Origin_index|Dest_index|            features|       features_norm|prediction|
+-----+----------+---------+-------+----------+-------+----------+----------+--------------+--------+--------+---------+-------------------+-------------+------------+----------+--------------------+--------------------+----------+
|    1|         1|        2|    2.0|      2255|   57.0|      2357|07/31/2001|          62.0|       0|   

In [29]:
# kmeans
input_columns_kmeans = [col for col in df.columns if (col != "ArrDelay" and col != "IssueDate")]
k = 3
kmeans_model = estimate_kmeans(df, input_columns_kmeans, k)

+-----+----------+---------+-------+----------+-------+----------+----------+--------------+--------+--------+---------+-------------------+-------------+------------+----------+--------------------+----------+
|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime| IssueDate|CRSElapsedTime|ArrDelay|DepDelay|Cancelled|UniqueCarrier_index|TailNum_index|Origin_index|Dest_index|            features|prediction|
+-----+----------+---------+-------+----------+-------+----------+----------+--------------+--------+--------+---------+-------------------+-------------+------------+----------+--------------------+----------+
|    1|         1|        2|    2.0|      2255|   57.0|      2357|07/31/2001|          62.0|       0|    67.0|        0|               11.0|        685.0|         0.0|      85.0|[1.0,1.0,2.0,2.0,...|         0|
|    1|         1|        2|    3.0|      2110|  323.0|        40|12/06/2000|         150.0|       0|   173.0|        0|                0.0|        529.0|  